<a href="https://colab.research.google.com/github/robinanwyl/oud_transcriptomics/blob/main/BENG204_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BENG 204 Project: Understanding Transcriptional Responses to Opioid Exposure Across Neurodevelopmental Stages in Brain Organoid Models

Mount the drive (run this cell every time the notebook is opened, and enable permissions if prompted)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# filepath for the project data is now "/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/"

Mounted at /content/drive


If the scanpy and anndata import statements cannot be resolved, run this cell to re-install those packages.

In [2]:
%pip install scanpy
%pip install anndata
%pip install pydeseq2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 859.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 5.6 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=cb78a884eb05eb5cf3ab993caa28e9f65caca64610fe523588519796c8161a23
  Stored in directory: /root/.cache/pip/wheels/4e/56/35/a748fc57279a4b

Import statements

In [3]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
# import matplotlib.pyplot as plt
# etc.

## Read in Kim et al scRNA-seq 10X output files and save as .h5ad.gz files (perform once)
**Read in the scRNA-seq data for Kim et al dataset day 53 untreated sample and day 53 acute fentanyl treatment sample. Merge the datasets (using unique cell barcodes with sample identifiers). Save the two datasets and the merged dataset as `.h5ad.gz` files.**

The original sample IDs are KH001 for the day 53 untreated sample and KH002 for the day 53 acute fentanyl treatment sample. Each sample has its own folder containing 3 compressed (`.gz`) files, which are the 10X Genomics CellRanger output files:

*   `matrix.mtx.gz` is a count matrix where rows are single cells, columns are genes, and each cell is the read count of that gene in that cell
*   `barcodes.tsv.gz` contains the cell barcodes (each cell is labeled with a unique barcode, which is used as an identifier)
*   `features.tsv.gz` contains the gene names

For each sample, we will first use `scanpy.read_10x_mtx()` to read the 3 files into a single `AnnData` object that contains the cell-by-gene matrix and associated metadata (barcodes and features):

In [ ]:
# sample1_path = "/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/Kim_KH001_Day53_Untreated"
# adata1 = sc.read_10x_mtx(sample1_path, var_names="gene_symbols", cache=True)
# adata1.obs["sample"] = "kim_day53_untreated"

# sample2_path = "/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/Kim_KH002_Day53_FTY_Acute"
# adata2 = sc.read_10x_mtx(sample2_path, var_names="gene_symbols", cache=True)
# adata2.obs["sample"] = "kim_d53_fty_acute"

Now we will merge the two samples into one AnnData object. Since we want to keep track of which sample is which, we will first prepend the cell barcodes with a sample description: "d53_ut" for the untreated samples and "d53_fty" for the treated samples.

In [ ]:
# adata1.obs.index = [f"d53_ut_{barcode}" for barcode in adata1.obs.index]
# adata2.obs.index = [f"d53_fty_{barcode}" for barcode in adata2.obs.index]
# adata_combined = sc.concat([adata1, adata2], label="batch", keys=["sample1", "sample2"])
# print("day 53 samples merged\n", adata_combined)

day 53 samples merged
 AnnData object with n_obs × n_vars = 10630 × 33538
    obs: 'sample', 'batch'


Now we will save the three AnnData objects as `.h5ad.gz` files for later use.

In [ ]:
# adata1.write("/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/kim_d53_ut.h5ad.gz", compression="gzip")
# adata2.write("/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/kim_d53_fty.h5ad.gz", compression="gzip")
# adata_combined.write("/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/kim_d53_combined.h5ad.gz", compression="gzip")

## Read in Kim et al .h5ad.gz files

Now the Kim et al datasets that were saved as .h5ad.gz files can be read in as AnnData objects using `scanpy.read_h5ad()`.

In [ ]:
kim_d53_ut = sc.read_h5ad("/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/kim_d53_ut.h5ad.gz")
kim_d53_fty = sc.read_h5ad("/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/kim_d53_fty.h5ad.gz")
kim_d53 = sc.read_h5ad("/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/kim_d53_combined.h5ad.gz")

## Read in Ho et al files

In [5]:
counts_raw = pd.read_table("/content/drive/My Drive/BENG204_Project/BENG204_Project_Data/GSE210206_counts.txt", )
counts_raw.head()
# transpose
# filter
# blah

,gene_symbol,OFO1003B1,OFO1003B2,OFO1003O1,OFO1003O2,OFO1003V1,OFO1003V2,OFO1004B1,OFO1004B2,OFO1004O1,OFO1004O2,OFO1004V1,OFO1004V2,OFO1005B1,OFO1005B2,OFO1005O1,OFO1005O2,OFO1005V1,OFO1005V2
0,DDX11L1,1,0,0,0,0,0,0,2,2,0,0,0,1,0,0,0,0,1
1,WASH7P,448,437,412,510,637,686,263,300,247,329,256,163,373,367,182,222,252,242
2,MIR6859-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,MIR6859-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,FAM138A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
